# CH12 설문 데이터 분석과 지도 시각화

업로드된 **CH12 설문 데이터 분석과 지도 시각화.ppt** 내용을 바탕으로,  
1) 구글 설문 결과(스프레드시트) 연동 흐름, 2) 설문 데이터 전처리/분석, 3) folium 지도 시각화, 4) geopy 지오코딩,  
그리고 **[플러스 예제] 관광지 방문자 수 지도** 형태로  
👉 **설명 + 실습 예제**로 구성한 노트북입니다. fileciteturn8file0

> ⚠️ 참고: 이 노트북은 **Jupyter 환경**에서 실행 가능한 형태로 작성되어 있으며,  
> 구글 스프레드시트 연동(코랩 인증/권한 등)은 환경에 따라 일부 셋업이 필요합니다.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 12.1 구글 설문 결과 연동(개념)

PPT에서는 Google Form → 응답 → Google Sheets로 저장 후, 코랩에서 스프레드시트를 불러오는 흐름을 설명합니다. fileciteturn8file0

여기서는 **2가지 방식**을 제시합니다.

- **A안(권장, 로컬/Jupyter)**: 응답 시트를 CSV로 내려받아 `pd.read_csv()`로 분석
- **B안(코랩)**: 인증 후 `gspread`/`google.colab`을 이용해 시트를 바로 읽기

### A안) CSV로 내려받아 읽기 (가장 안정적)

In [ ]:
# 파일명만 바꿔서 사용하세요.
# df = pd.read_csv("responses.csv")
# df.head()

print("A안: responses.csv 같은 파일을 준비하면 read_csv로 바로 분석할 수 있어요.")


### B안) 코랩에서 구글 시트 직접 읽기 (개념 코드)
> 코랩 환경에서만 바로 작동하는 경우가 많습니다. (인증/권한 필요)

In [ ]:
# (코랩 예시) 환경에 따라 설치/인증이 필요합니다.
# !pip -q install gspread
# from google.colab import auth
# auth.authenticate_user()
# import gspread
# from google.auth import default
# creds, _ = default()
# gc = gspread.authorize(creds)
# sh = gc.open_by_url("구글시트URL")
# ws = sh.sheet1
# df = pd.DataFrame(ws.get_all_records())
# df.head()

print("B안: 코랩에서 인증 후 gspread로 시트를 읽을 수 있습니다(개념 코드).")


## 12.1-2 설문 데이터 전처리/분석 예제
PPT의 예시는 지원자 정보(지원자번호, 희망업무(복수), 자격증 점수, 토익 점수)를 분석합니다. fileciteturn8file0

아래는 **샘플 설문 데이터**로 동일한 흐름을 실습합니다.

In [ ]:
df = pd.DataFrame({
    "타임스탬프": pd.date_range("2025-01-01", periods=8, freq="D"),
    "지원자번호": [101,102,103,104,105,106,107,108],
    "희망 업무(복수 가능)": ["개발, 기획","개발","영업","경영, 기획","개발, 영업","기획","개발, 경영","개발, 기획, 경영"],
    "자격증 점수(기사:1000,산업기사:900,기능사:800)": ["1000","900","800","900","1000","800","900","1000"],
    "토익 점수": ["850","780","650","900","720","810","770","880"]
})
df.head()


### (1) 열 이름 줄이기 + 숫자형 변환 + 타임스탬프 삭제

In [ ]:
df2 = df.rename(columns={
    "자격증 점수(기사:1000,산업기사:900,기능사:800)":"자격증 점수",
    "희망 업무(복수 가능)":"희망 업무"
}).copy()

df2["자격증 점수"] = df2["자격증 점수"].astype(int)
df2["토익 점수"] = df2["토익 점수"].astype(int)
df2 = df2.drop(columns=["타임스탬프"])

df2


### (2) 합계/합격 판정 열 만들기
PPT에서는 합계가 기준 이상이면 합격으로 판별합니다. fileciteturn8file0

In [ ]:
df2["합계"] = df2["자격증 점수"] + df2["토익 점수"]
df2["결과"] = np.where(df2["합계"] >= 1500, "합격", "불합격")
df2


### (3) 합격자만 보기 + CSV 저장

In [ ]:
pass_df = df2[df2["결과"] == "합격"]
pass_df


In [ ]:
out_csv = "result.csv"
df2.to_csv(out_csv, index=False)
out_csv


### (4) 희망 업무 빈도수 구하기 (split 활용)
복수 선택 응답은 쉼표로 구분되어 있으므로 split해서 카운트합니다. fileciteturn8file0

In [ ]:
desired_work = {}
for s in df2["희망 업무"]:
    for w in str(s).split(","):
        w = w.strip()
        desired_work[w] = desired_work.get(w, 0) + 1

desired_work


In [ ]:
# 막대 그래프
labels = list(desired_work.keys())
values = list(desired_work.values())

ind = np.arange(len(labels))
plt.bar(ind, values)
plt.xticks(ind, labels)
plt.title("희망 업무 빈도수")
plt.show()


## LAB) 키/몸무게 설문으로 비만도(BMI) 분석
PPT의 LAB 흐름: 키&몸무게 설문 → BMI 계산 → 분류. fileciteturn8file0

In [ ]:
lab = pd.DataFrame({
    "이름": ["A","B","C","D","E"],
    "키(cm)": [170, 160, 175, 168, 182],
    "몸무게(kg)": [65, 58, 92, 75, 80]
})
lab["키(m)"] = lab["키(cm)"] / 100
lab["BMI"] = lab["몸무게(kg)"] / (lab["키(m)"]**2)

# BMI 분류(간단 기준)
lab["판정"] = np.where(lab["BMI"] < 18.5, "저체중",
               np.where(lab["BMI"] < 23, "정상",
               np.where(lab["BMI"] < 25, "과체중", "비만")))

lab


## 12.2 지도 시각화 (folium)
PPT에서는 `folium.Map`으로 지도를 생성하고 마커/툴팁/팝업, 타일(배경), 확대, 저장을 다룹니다. fileciteturn8file0

In [ ]:
# folium이 설치되어 있지 않으면 설치가 필요할 수 있어요.
# !pip -q install folium

import folium


In [ ]:
# 서울시청 중심 지도
m = folium.Map(location=[37.566679, 126.978291], zoom_start=12, tiles="OpenStreetMap")
m


In [ ]:
# 마커 추가: 경복궁(예시)
folium.Marker(
    location=[37.579754, 126.976681],
    tooltip="경복궁",
    popup="연락처: 02-3700-3900"
).add_to(m)

m


In [ ]:
# 저장(HTML)
m.save("seoul_map.html")
"seoul_map.html"


## 12.2-2 여러 마커를 DataFrame으로 추가하기 (고궁 4곳 예시)
PPT의 핵심: DataFrame을 `iterrows()`로 돌며 마커를 추가합니다. fileciteturn8file0

In [ ]:
palace = pd.DataFrame({
    "이름": ["경복궁","창덕궁","창경궁","덕수궁"],
    "위도": [37.579754, 37.579430, 37.578840, 37.565804],
    "경도": [126.976681, 126.991042, 126.994667, 126.975146]
})

m2 = folium.Map(location=[37.566679, 126.978291], zoom_start=12)

for _, row in palace.iterrows():
    folium.Marker(
        location=[row["위도"], row["경도"]],
        tooltip=row["이름"]
    ).add_to(m2)

m2


## 12.2-3 지오코딩(geopy)로 주소/지명 → 위도/경도
PPT에서는 `geopy.geocoders.Nominatim`의 `geocode()`로 위경도를 얻습니다. fileciteturn8file0

In [ ]:
# !pip -q install geopy
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="my_geo_app")

place = "경복궁"
loc = geolocator.geocode(place)
if loc:
    print(loc.address)
    print(loc.latitude, loc.longitude)
else:
    print("지오코딩 실패: 네트워크/요청 제한/환경 설정을 확인하세요.")


## 12.3 [플러스 예제] 관광지 방문자 수 지도
관광지별 위경도 + 방문자 수를 준비한 뒤, 방문자 수에 따라 마커 색/크기를 다르게 표현할 수 있습니다. fileciteturn8file0

In [ ]:
tour = pd.DataFrame({
    "관광지": ["A해변", "B오름", "C박물관", "D시장"],
    "위도": [33.450, 33.370, 33.515, 33.500],
    "경도": [126.570, 126.690, 126.530, 126.520],
    "방문자수": [1200, 800, 2200, 1500]
})

center = [tour["위도"].mean(), tour["경도"].mean()]
m3 = folium.Map(location=center, zoom_start=11)

# 방문자수에 따라 원 크기(반지름) 조절
for _, r in tour.iterrows():
    folium.CircleMarker(
        location=[r["위도"], r["경도"]],
        radius=max(4, min(20, r["방문자수"]/150)),  # 간단 스케일
        tooltip=r["관광지"],
        popup=f"방문자수: {r['방문자수']}",
        fill=True
    ).add_to(m3)

m3


## 정리 체크리스트
- [ ] 설문 데이터를 불러와 DataFrame으로 만들 수 있다.
- [ ] rename/astype/drop, 합계/판정 열 생성, 필터링을 할 수 있다.
- [ ] 복수응답을 split하여 빈도수를 구할 수 있다.
- [ ] folium Map 생성, 마커/팝업/툴팁 추가, HTML 저장을 할 수 있다.
- [ ] geopy로 지오코딩하여 위경도를 얻고 지도에 반영할 수 있다.
